In [1]:
import pandas as pd
import os
import csv
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
%matplotlib
import matplotlib.lines as mlines
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [2]:
from Experimentos2 import Experimentos, labelLines

[Errno 2] No such file or directory: '.\\Data\\5cc50C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - max av.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - fspot.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - max av cup.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - max av.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - fspot.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - max av cup.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - max av.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - fspot.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc

In [3]:
velos = pd.read_excel('Velocidades y punto.xlsx')
PropTerm = pd.read_excel('PropiedadesTermicas.xlsx')

In [4]:
numeropixeles = 1020100
rho_manjar = 1340
kilos_manjar = 2

In [5]:
#genero ESPESORES a partir de AREAS aproximando el flujo a un ELIPSOIDE REAL. 
for exp in Experimentos:
    if exp.flir is not None:
        try:
            exp.flir['area_m2'] = exp.flir['FLIR: area']/numeropixeles
            exp.flir['volumen_emitido'] = exp.param['caudal'][0]*exp.flir['FLIR: tiempo']
        except:
            continue
        volumen_total = kilos_manjar/rho_manjar
        exp.flir['volumen_emitido'][exp.flir['volumen_emitido'] > volumen_total] = volumen_total
        exp.flir['espesor area'] = 3*exp.flir['volumen_emitido']/(2*exp.flir['area_m2'])
        exp.dinamica['area a*b'] = 3.14*(exp.dinamica['avance: largo total flujo']/2)*(exp.dinamica['avance: ancho maximo']/2)
        exp.dinamica['volumen a*b'] = exp.param['caudal'][0]*exp.dinamica['avance: tiempo']
        exp.dinamica['volumen a*b'][exp.dinamica['volumen a*b'] > volumen_total] = volumen_total
        exp.dinamica['espesor largoancho'] = 3*exp.dinamica['volumen a*b']/(2*exp.dinamica['area a*b'])

C:\Users\CCVV\Anaconda3b\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\CCVV\Anaconda3b\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
for exp in Experimentos:
    exp.param['tiempo fin eyeccion'] = exp.param['volumen total teorico']/exp.param['caudal']

In [7]:
#propiedades medidas a temperatura ambiente
Conduct = PropTerm['Conductividad'].mean()
Cap = PropTerm['Capacidad'].mean()*10**6
Difus = PropTerm['Difusividad'].mean()*10**(-7)
Resist = PropTerm['Resistividad'].mean()
Cp = Cap/1340

In [8]:
def YieldStg(temp):
    return 6220*temp**(-1.36)
def Konsist(temp):
    return 1*10**6 * temp**(-2.61)
def FlowInd(temp):
    return 0.0046*temp + 0.145

In [9]:
for exp in Experimentos:
    #updateo densidad
    exp.param['densidad']=1340
    #calculo cos angulo
    exp.param['slope sen'] = np.sin(np.radians(exp.param['angulo inclinacion']))
    exp.param['slope cos'] = np.cos(np.radians(exp.param['angulo inclinacion']))

In [10]:
#pd.set_option('display.max_columns', None)
#Experimentos[7].param.head()

In [10]:
#Añado velos a params x facilidad
for exp in Experimentos:
    for i in range(33):
        if exp.param['NOMBRE'][0]==velos.iloc[i,:]['nombre']:
            exp.param['v1'] = velos.iloc[i,:]['v1']
            exp.param['v2'] = velos.iloc[i,:]['v2']
            exp.param['L*'] = velos.iloc[i,:]['L*']
            exp.param['t*'] = velos.iloc[i,:]['t*']

In [11]:
#Estimo Jeffreys parte primera
for exp in Experimentos:
    if exp.param['NOMBRE'][0] == '1cc48C':
        continue
    exp.param['numerador'] = 9.8*exp.param['densidad'][0]*exp.param['slope sen'][0]*(exp.perfil['perfil: espesor'].max())**2
    if exp.param['v1'][0] > 0:
        exp.param['denominador1'] = 3*exp.param['v1'][0]
    if exp.param['v2'][0] > 0:
        exp.param['denominador2'] = 3*exp.param['v2'][0]

In [12]:
#Estimo Jeffreys parte segunda y final
for exp in Experimentos:
    if exp.param['NOMBRE'][0] == '1cc48C':
        continue
    if exp.param['v1'][0] > 0:
        exp.param['visc1'] = exp.param['numerador'][0]/exp.param['denominador1'][0]
    if exp.param['v2'][0] > 0:
        exp.param['visc2'] = exp.param['numerador'][0]/exp.param['denominador2'][0]
        #print(exp,exp.param['visc2'])

In [13]:
#Largo según Lister para flujos newtonianos
for exp in Experimentos:
    exp.dinamica['numerador'] = exp.param['caudal'][0]*exp.param['caudal'][0]*exp.param['densidad'][0]*9.8*exp.param['slope sen'][0]
    if exp.param['v1'][0] > 0:
        exp.dinamica['denominador'] = exp.param['visc1'][0]*exp.dinamica['avance: ancho maximo']*exp.dinamica['avance: ancho maximo']
        exp.dinamica['division'] = np.cbrt(exp.dinamica['numerador'][0]/exp.dinamica['denominador'])
        exp.dinamica['Lteonew'] = exp.dinamica['division']*exp.dinamica['avance: tiempo']

In [14]:
#Calculo y Guardo el Bingham para todos los exp
for exp in Experimentos:
    temp = exp.param['t max'][0]
    exp.param['Ty'] = YieldStg(temp)
    exp.param['K'] = Konsist(temp)
    exp.param['FlowI'] = FlowInd(temp)
    exp.param['H'] = exp.perfil['perfil: espesor'].max()
    exp.param['Lmax'] = exp.dinamica['avance: distancia desde punto eyeccion'].max()
    exp.param['e'] = exp.param['H'][0]/exp.param['Lmax'][0]
    #Bingham B=tp/e*rho*g*H*cos(o) ec (2.6) shallow viscoplastic flow on an inclined plane de balmforth
    exp.param['Bingham'] = exp.param['Ty'][0] / (exp.param['e'][0]*exp.param['densidad'][0]*9.8*exp.param['H'][0]*exp.param['slope cos'][0])
    #B2 y B3 bingham 2 ec 3.12 cooling domes of viscoplastic fluids
    if exp.param['v1'][0] > 0:
        exp.param['B2'] = exp.param['Ty'][0]*exp.param['H'][0]/(exp.param['visc1'][0]*exp.param['v1'][0])
    exp.param['B3'] = exp.param['Ty'][0]*exp.param['Lmax'][0]/(exp.param['densidad'][0]*9.8*exp.param['H'][0]**2)
#seconda universita de napoles

In [73]:
##cosas necesarias para el ploteo

In [16]:
#FUNCION Q DEVUELVE RANGO [1-7] SEGUN TEMP MAXIMA DEL EXP
def rango_temperatura(temp):
    if temp < 40:
        return 1
    if temp < 45:
        return 2
    if temp < 50:
        return 3
    if temp < 55:
        return 4
    if temp < 60:
        return 5
    if temp < 65:
        return 6
    if temp < 70:
        return 7

In [17]:
listalargonewtoniano = []

for exp in Experimentos:
    holder = {}
    holder['caudal'] = exp.param['caudal'][0]
    holder['angulo'] = exp.param['angulo inclinacion'][0]#en grados
    holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])
    holder['temperatura'] = exp.param["t max"][0]
    holder['v1'] = exp.param['v1'][0]
    holder['tiempo'] = exp.dinamica['avance: tiempo']
    holder['distancia'] = exp.dinamica['avance: distancia desde punto eyeccion']
    holder['nombre'] = exp.param['NOMBRE'][0]

    if exp.param['v1'][0] > 0:
        holder['newtoniano'] = exp.dinamica['Lteonew']
        #holder['teyec'] = exp.param['tiempo fin eyeccion']

    listalargonewtoniano.append(holder)

In [18]:
#se arman GRUPOS SEGUN CAUDAL
from itertools import groupby
count = 0

for key, group in groupby(sorted(listalargonewtoniano, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    count += 1

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [19]:
#GENERA PATCHES DE COLORES SEGUN TEMPERATURA
import matplotlib.patches as mpatches

pat1 = mpatches.Patch(color='C1', label='temp < 40')
pat2 = mpatches.Patch(color='C2', label='temp < 45')
pat3 = mpatches.Patch(color='C3', label='temp < 50')
pat4 = mpatches.Patch(color='C4', label='temp < 55')
pat5 = mpatches.Patch(color='C5', label='temp < 60')
pat6 = mpatches.Patch(color='C6', label='temp < 65')
pat7 = mpatches.Patch(color='C7', label='temp < 70')
pat8 = mpatches.Patch(color='black', label='15° slope')

numerocolores = 7
colors = ['pat{}'.format(i+1) for i in range(numerocolores)] + [pat8]

print(colors)

['pat1', 'pat2', 'pat3', 'pat4', 'pat5', 'pat6', 'pat7', <matplotlib.patches.Patch object at 0x000002CAAF178780>]


In [ ]:
#Empieza el ploteo

In [25]:
# [LISTO] plot listalargonewtoniano para los 33 experimentos
fig = plt.figure()
axes = []
f = 0

for x in range(32):
    axes.append(fig.add_subplot(7, 5, x + 1))

for i, (key, group) in enumerate(groupby(sorted(listalargonewtoniano, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(i, key)
    
    for exp in sorted(group, key=lambda x: x['temperatura']):
    
        #excepciones
        if exp['nombre'] == '1cc48C':
            continue
            
        axes[f].grid(True, which='both', color='gainsboro')
    
        axes[f].set_title(exp['nombre'])

        if exp['angulo'] == 15:
            axes[f].scatter(exp['tiempo'], exp['distancia'], color='m', s=2, marker='o')
            axes[f].plot(exp['tiempo'], exp['newtoniano'], color='b', linestyle=':')
        else:
            axes[f].scatter(exp['tiempo'], exp['distancia'], color='r', marker= 'o', s=2)
            if exp['v1'] > 0:
                axes[f].plot(exp['tiempo'], exp['newtoniano'], color='b', linestyle='--')

        f = f+1
        #print(f)

plt.suptitle('Aproximacion avance de flujo viscoso según Lister(1992)')        
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
#labelLines(plt.gca().get_lines(),align=False,color='k')
fig.show()

0 1e-06
1 5e-06
2 1e-05
3 1.7e-05
4 2.5e-05


In [21]:
#Creo la funcion del ajuste 1 con curve_fit y pruebo con un plot

def func(x, k, c):
    return c*x**k

exp = Experimentos[7]

xData = exp.dinamica['avance: tiempo']
yData = exp.dinamica['avance: distancia desde punto eyeccion']

popt, pcov = curve_fit(func, xData, yData)

exp.dinamica['nueva wea'] = func(exp.dinamica['avance: tiempo'], *popt)
exp.dinamica['k'] = popt[0]
exp.dinamica['c'] = popt[1]

## PLOTEO DE DATOS

fig = plt.figure()
k = exp.dinamica['k'][0]
c = exp.dinamica['c'][0]
textstr = '$k=%f$\n$c=%f$'%(k,c)

plt.scatter(exp.dinamica['avance: tiempo'], exp.dinamica['avance: distancia desde punto eyeccion'], 
            color='m', s=8, marker='o', label='Datos Experimentales')
plt.plot(exp.dinamica['avance: tiempo'], exp.dinamica['nueva wea'], color='b', linestyle=':', label='Ajuste teórico')
plt.ylabel('Distancia [m]')
plt.xlabel('Tiempo [s]')
plt.suptitle('Dinámica de flujo{}'.format(exp.param['NOMBRE'][0]))
plt.figtext(0.7, 0.4, textstr, verticalalignment='top')

datos = mlines.Line2D([], [], color='m', linestyle='', marker='o',markersize=4, label='Datos Experimentales')
ajuste = mlines.Line2D([], [], color='b', linestyle=':', marker='',markersize=4, label='Ajuste Teorico')
ecuacion = mlines.Line2D([], [], color='b', linestyle='', marker='',markersize=4, label=r'$\c*t^k$')


plt.legend(handles=[datos, ajuste, ecuacion], title='Leyenda', loc=1, bbox_to_anchor=(0.5, 0.3, 0.4, 0.4), 
           labels=['Datos Experimentales', 'Ajuste Teorico', r'$c*t^k$'], ncol=1)

fig.show()

In [19]:
#guardo los valores de AMBOS ajustes (pot y log) para cada exp en dinamica pra TODOS los datos (1 linea)
def func(x, k, c):
    return c*x**k

for exp in Experimentos:

    xData = exp.dinamica['avance: tiempo']
    yData = exp.dinamica['avance: distancia desde punto eyeccion']
    xDatalog = np.log10(exp.dinamica['avance: tiempo'][1:]) #elimino los valores iguales a 0
    yDatalog = np.log10(exp.dinamica['avance: distancia desde punto eyeccion'][1:]) #elimino los valores iguales a 0

# Calculate linear reggresion for log
# log10(yData) = a + b*log10(xData) // donde 10^intercept*xData^slope es equivalente a a*xData^b
    slope, intercept, r_value, p_value, slope_std_error = stats.linregress(xDatalog, yDatalog)
    exp.dinamica['eclog'] = intercept + slope * xDatalog #ecuacion lineal en caso de plotear en plt.loglog
    exp.dinamica['fitlog'] = 10**intercept *xData**slope
    exp.dinamica['a'] = 10**intercept
    exp.dinamica['b'] = slope

# Calculate curve_fit reggresion for data
    popt, pcov =  curve_fit(func, exp.dinamica['avance: tiempo'], exp.dinamica['avance: distancia desde punto eyeccion'],
                            diag=(1./exp.dinamica['avance: tiempo'].mean(),
                                  1./exp.dinamica['avance: distancia desde punto eyeccion'].mean()))
    exp.dinamica['fitpot'] = func(xData, *popt)
    k = popt[0]
    c = popt[1]
    exp.dinamica['k'] = popt[0]
    exp.dinamica['c'] = popt[1]
    #print(pcov)
    #exp.dinamica['r2'] = pcov[0]

In [20]:
#NUEVA LISTA para valores pot y log sin dividir los datos por punto.

listafuncionespotlog = []

for exp in Experimentos:
    holder = {}
    holder['caudal'] = exp.param['caudal'][0]
    holder['angulo'] = exp.param['angulo inclinacion'][0]#en grados
    holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])
    holder['temperatura'] = exp.param["t max"][0]
    holder['v1'] = exp.param['v1'][0]
    holder['tiempo'] = exp.dinamica['avance: tiempo']
    holder['distancia'] = exp.dinamica['avance: distancia desde punto eyeccion']
    holder['fitpot'] = exp.dinamica['fitpot']
    holder['fitlog'] = exp.dinamica['fitlog']
    holder['nombre'] = exp.param['NOMBRE'][0]
    holder['k'] = exp.dinamica['k'][0]
    holder['c'] = exp.dinamica['c'][0]
    holder['a'] = exp.dinamica['a'][0]
    holder['b'] = exp.dinamica['b'][0]

    #if exp.param['v1'][0] > 0:
         #holder['newtoniano'] = exp.dinamica['Lteonew']
         #holder['teyec'] = exp.param['tiempo fin eyeccion']

    listafuncionespotlog.append(holder)

In [22]:
# [LISTO] plot listafuncionespotlog los 33 experimentos con ajuste potencial sin escala log.

fig = plt.figure()
axes = []
f = 0

for x in range(33):
    axes.append(fig.add_subplot(7, 6, x + 1))

for i, (key, group) in enumerate(groupby(sorted(listafuncionespotlog, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(i, key)

    for exp in sorted(group, key=lambda x: x['temperatura']):
        k = exp['k']
        c = exp['c']
        fin = exp['tiempo'].iloc[-1]
        dist = exp['distancia'].iloc[-1]
        axes[f].set_title(exp['nombre'])
        textstr = '$k=%.3f$\n$c=%.3f$'%(k,c)
        axes[f].set_ylabel('Distancia [m]')
        #axes[f].set_xlabel('Tiempo [s]')
        axes[f].text(fin/1.5, dist*0.1, textstr)
            
        if exp['angulo'] == 15:
            axes[f].scatter(exp['tiempo'], exp['distancia'], color='m', s=2, marker='o')
            axes[f].plot(exp['tiempo'], exp['fitpot'], color='b', linestyle=':')

        else:
            axes[f].scatter(exp['tiempo'], exp['distancia'], color='r', marker= 'o', s=2)
            #if exp['v1'] > 0:
            axes[f].plot(exp['tiempo'], exp['fitpot'], color='b', linestyle='--')

        f = f+1
        
        #print(f)
        #print(i, exp['nombre'])

        
datos = mlines.Line2D([], [], color='m', linestyle='', marker='o',markersize=4, label='Datos Experimentales')
ajuste = mlines.Line2D([], [], color='b', linestyle=':', marker='',markersize=4, label='Ajuste Teorico')
ecuacion = mlines.Line2D([], [], color='b', linestyle='', marker='',markersize=4, label=r'$\c*t^k$')

plt.legend(handles=[datos, ajuste, ecuacion], title='Leyenda', loc=0, bbox_to_anchor=(3, 1, 0.4, 0.4), 
           labels=['Datos Experimentales', 'Ajuste Teorico', r'$c*t^k$'], ncol=1)

fig.show()


0 1e-06
1 5e-06
2 1e-05
3 1.7e-05
4 2.5e-05


In [26]:
# Calculate curve_fit reggresion for data
def func(x, k, c):
    return c*x**k

In [24]:
#plot listafuncionespotlog todos los exp de 1 caudal a la vez
# 1 cc 10 exp 5 cc 8 exp 10cc 1e-05 8 exp 25cc 2.5e-05 6 exp 5a25xx 1 exp

fig = plt.figure()
axes = []
f = 0

for x in range(10): #range segun el numero de exp por caudal
    axes.append(fig.add_subplot(4, 3, x + 1)) #subplot segun numero de figuras en el plot (3x3, 4x3, 2x3 etc)

for i, (key, group) in enumerate(groupby(sorted(listafuncionespotlog, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(i, key)
    if key==1e-06:
        for exp in sorted(group, key=lambda x: x['temperatura']):
            k = exp['k']
            c = exp['c']
            a = exp['a']
            b = exp['b']
            fin = exp['tiempo'].iloc[-1]
            dist = exp['distancia'].iloc[-1]

            axes[f].set_title(exp['nombre'])
            textstr = '$\star k=%f$ \n$c=%f$ \n$\star b=%f$ \n$a=%f$'%(k,c,b,a)

            axes[f].set_ylabel('Distancia [m]')
            axes[f].set_xlabel('Tiempo [s]')
            axes[f].set_title('{}'.format(exp['nombre']))
            axes[f].text(fin/1.5, dist*0.05, textstr)

            if exp['angulo'] == 15:
                axes[f].scatter(exp['tiempo'], exp['distancia'], color='m', marker='o')
                axes[f].plot(exp['tiempo'], exp['fitlog'], color='b', linestyle=':')
                axes[f].plot(exp['tiempo'], exp['fitpot'], color='g', linestyle=':')

            else:
                axes[f].scatter(exp['tiempo'], exp['distancia'], color='m', marker= 'o')
                axes[f].plot(exp['tiempo'], exp['fitlog'], color='b', linestyle='--')
                axes[f].plot(exp['tiempo'], exp['fitpot'], color='g', linestyle=':')

            f = f+1
            #print(f)
            #print(i, exp['nombre'])
        
datos = mlines.Line2D([], [], color='m', linestyle='', marker='o',markersize=4, label='Datos Experimentales')
ajustelog = mlines.Line2D([], [], color='b', linestyle=':', marker='',markersize=4, label='Ajuste Teorico logaritmico')
ecuacion1 = mlines.Line2D([], [], color='b', linestyle='', marker='',markersize=4, label=r'$\a*t^b$')
ajustepot = mlines.Line2D([], [], color='g', linestyle=':', marker='',markersize=4, label='Ajuste Teorico potencial')
ecuacion2 = mlines.Line2D([], [], color='b', linestyle='', marker='',markersize=4, label=r'$\c*t^k$')

plt.legend(handles=[datos, ajustelog, ecuacion1, ajustepot, ecuacion2], title='Leyenda', loc=1, bbox_to_anchor=(1.4, 0.5, 0.4, 0.4), 
           labels=['Datos Experimentales', 'Ajuste Teorico logaritmico',r'$a*t^b$', 'Ajuste Teorico potencial', r'$c*t^k$'], ncol=1)

fig.show()

0 1e-06
1
0 1cc24C
2
0 1cc47C
3
0 1cc48C
4
0 1cc51C
5
0 1cc52C
6
0 1cc54C
7
0 1cc55C
8
0 1cc54,7C
9
0 1cc59C
10
0 1cc62C
1 5e-06
2 1e-05
3 1.7e-05
4 2.5e-05


In [29]:
#ERRORES SIN ARREGLAR AUN PARA GRAFICOS INDIVIDUALES, PLOTEO DE TODOS LOS EXP SI. 
#-> arreglados en lista para plotear x grupo caudal
#pruebo la funcion que divide los datos en L*, t* con un plot de datos 

#exp30 tiene 3 regimenes
#exp29 tiene 3 regimenes
#exp25 tiene 3 regimenes
#exp23 tiene 3 regimenes
#exp22 tiene 3 regimenes, 3o con 2 datos. 1cc24C

#exp24 ajuste linea azul mejor q verde
#exp20 ajuste linea verde mejor q azul
#exp15 linea azul mejor ajuste q verde
#exp8 bien fea curva

#exp27 error: inputs must not be empty
#exp16 ---> 53 prDespues = curve_fit(func, xDataB, yDataB) TypeError: Improper input: N=2 must not exceed M=1

#exp6 --> 36 lnrDespues = stats.linregress(xDataLogB, yDataLogB) inputs must not be empty
#exp5 ---> 56 prDespues = curve_fit(func, xDataB, yDataB) Improper input: N=2 must not exceed M=1
#exp1 ---> 38 lnrDespues = stats.linregress(xDataLogB, yDataLogB) inputa must not be empty

exp = Experimentos[26]

xData = exp.dinamica['avance: tiempo']
yData = exp.dinamica['avance: distancia desde punto eyeccion']
yDataymed = exp.dinamica['avance: ancho maximo']/2
yDataymax = exp.dinamica['avance: ancho maximo']
yDataxmin = exp.dinamica['avance: distancia hacia atrás']


antes = exp.dinamica[exp.dinamica['avance: tiempo'] < exp.param['t*'][0]]
despues = exp.dinamica[exp.dinamica['avance: tiempo'] > exp.param['t*'][0]]

xDataLogA = np.log10(antes['avance: tiempo'][1:])
xDataLogB = np.log10(despues['avance: tiempo'])
yDataLogA = np.log10(antes['avance: distancia desde punto eyeccion'][1:])
yDataLogB = np.log10(despues['avance: distancia desde punto eyeccion'])

#ESTO en el plot para todos los datos para que no se caigan los exp que anote arriba con problemas
#if yDataLogB.shape[0] < 2:
#    continue
#if yDataLogA.shape[0] < 2:
#    continue
        
xDataA = antes['avance: tiempo']
xDataB = despues['avance: tiempo']
yDataA = antes['avance: distancia desde punto eyeccion']
yDataB = despues['avance: distancia desde punto eyeccion']

# Calculate linear reggresion for log
# log10(yData) = a + b*log10(xData) // donde 10^intercept*xData^slope es equivalente a a*xData^b
lnrAntes = stats.linregress(xDataLogA, yDataLogA)
lnrDespues = stats.linregress(xDataLogB, yDataLogB)

fitlogA = lnrAntes.intercept + lnrAntes.slope * xDataLogA #ecuacion lineal en caso de plotear en plt.loglog
fitlogB = lnrDespues.intercept + lnrDespues.slope * xDataLogB #ecuacion lineal en caso de plotear en plt.loglog

aA = 10**lnrAntes.intercept
bA = lnrAntes.slope

aB = 10**lnrDespues.intercept
bB = lnrDespues.slope

# Calculate curve_fit reggresion for data
def func(x, k, c):
    return c*x**k

prAntes =  curve_fit(func, xDataA, yDataA)
prDespues = curve_fit(func, xDataB, yDataB)
#fitpotA = func(xDataA, prAntes[0], prAntes[1])
#fitpotB = func(xDataB, prDespues[0], prDespues[1])

kA = prAntes[0][0]
cA = prAntes[0][1]

kB = prDespues[0][0]
cB = prDespues[0][1]

#Ploteo de datos

fig = plt.figure()

plt.loglog(xData, yData, color='m', marker='o', markersize='3', label='Datos Experimentales')
plt.loglog(xData, yDataymed, color='g', linestyle='-.', label='Datos Experimentales')
#plt.loglog(xData, yDataymax, color='g', linestyle='--', label='Datos Experimentales')
plt.loglog(xData, yDataxmin, color='g', linestyle='--', label='Datos Experimentales')

plt.loglog(xDataA[1:], aA*xDataA[1:]**bA, color='b', linestyle=':', label='Ajuste teórico logaritmico')##
#plt.loglog(xDataA[1:], cA*xDataA[1:]**kA, color='g', linestyle=':', label='Ajuste teórico potencial')
plt.loglog(xDataB, aB*xDataB**bB, color='r', linestyle=':', label='Ajuste teórico logaritmico')##
#plt.loglog(xDataB, cB*xDataB**kB, color='c', linestyle=':', label='Ajuste teórico potencial')

plt.ylabel('log(Distancia [m])')
plt.xlabel('log(Tiempo [s])')
plt.suptitle('Ajustes potencial y logaritmico para {}'.format(exp.param['NOMBRE'][0]))

textstr = '$\star k_a=%f$ \n $c_a=%f$ \n $\star k_d=%f$ \n $c_d=%f$ \n $\star b_a=%f$\n $a_a=%f$\n $\star b_d=%f$\n $a_d=%f$'%(kA,cA, kB, cB, bA, aA, bB, aB)
plt.figtext(0.6, 0.3, textstr, verticalalignment='top')

datos = mlines.Line2D([], [], color='m', linestyle='', marker='o',markersize=4, label='Datos Experimentales')
ajustelog = mlines.Line2D([], [], color='b', linestyle=':', marker='',markersize=4, label='Ajuste Teorico logaritmico')
ecuacion1 = mlines.Line2D([], [], color='b', linestyle='', marker='',markersize=4, label=r'$\a*t^b$')
ajustepot = mlines.Line2D([], [], color='g', linestyle=':', marker='',markersize=4, label='Ajuste Teorico potencial')
ecuacion2 = mlines.Line2D([], [], color='b', linestyle='', marker='',markersize=4, label=r'$\c*t^k$')


plt.legend(handles=[datos, ajustelog, ecuacion1, ajustepot, ecuacion2], title='Leyenda', loc=1, 
           bbox_to_anchor=(0.5, 0.2, 0.4, 0.4), labels=['Datos Experimentales', 'Ajuste Teorico logaritmico',r'$a*t^b$', 
                                                        'Ajuste Teorico potencial', r'$c*t^k$'], ncol=1)

fig.show()

In [27]:
#Guardo valores en todos los experimentos para numeros pre L*, t* y post, con sus aproximaciones potenciales y logaritmicas.

for exp in Experimentos:

    xData = exp.dinamica['avance: tiempo']
    yData = exp.dinamica['avance: distancia desde punto eyeccion']

    antes = exp.dinamica[exp.dinamica['avance: tiempo'] < exp.param['t*'][0]]
    despues = exp.dinamica[exp.dinamica['avance: tiempo'] > exp.param['t*'][0]]
    
    xDataA = antes['avance: tiempo']
    xDataB = despues['avance: tiempo']
    yDataA = antes['avance: distancia desde punto eyeccion']
    yDataB = despues['avance: distancia desde punto eyeccion']

    xDataLogAntes = np.log10(antes['avance: tiempo'][1:])
    xDataLogDespues = np.log10(despues['avance: tiempo'])
    yDataLogAntes = np.log10(antes['avance: distancia desde punto eyeccion'][1:])
    yDataLogDespues = np.log10(despues['avance: distancia desde punto eyeccion'])
    
    exp.dinamica['yDataLogDespues'] = yDataLogDespues
    exp.dinamica['yDataLogAntes'] = yDataLogAntes

    if yDataLogAntes.shape[0] >= 2:
        exp.dinamica['xDataAntes'] = antes['avance: tiempo'].dropna()
        exp.dinamica['yDataAntes'] = antes['avance: distancia desde punto eyeccion'].dropna()

        lnrAntes = stats.linregress(xDataLogAntes, yDataLogAntes)

        fitlogA = lnrAntes.intercept + lnrAntes.slope * xDataLogAntes #ecuacion lineal en caso de plotear en plt.loglog

        exp.dinamica['cteAntes'] = 10**lnrAntes.intercept
        exp.dinamica['expAntes'] = lnrAntes.slope

        prAntes =  curve_fit(func, xDataA, yDataA)
        exp.dinamica['expotAntes'] = prAntes[0][0]
        exp.dinamica['ctepotAntes'] = prAntes[0][1]
    
    #ESTO en el plot para todos los datos para que no se caigan los exp que anote arriba con problemas
    if yDataLogDespues.shape[0] >= 2:
        
        exp.dinamica['xDataDespues'] = despues['avance: tiempo'].dropna()
        exp.dinamica['yDataDespues'] = despues['avance: distancia desde punto eyeccion'].dropna()

    # Calculate linear reggresion for log
    # log10(yData) = a + b*log10(xData) // donde 10^intercept*xData^slope es equivalente a a*xData^b

        lnrDespues = stats.linregress(xDataLogDespues, yDataLogDespues)

        fitlogB = lnrDespues.intercept + lnrDespues.slope * xDataLogDespues #ecuacion lineal en caso de plotear en plt.loglog

        exp.dinamica['cteDespues'] = 10**lnrDespues.intercept
        exp.dinamica['expDespues'] = lnrDespues.slope

    # Calculate curve_fit reggresion for data

        prDespues = curve_fit(func, xDataB, yDataB)
        #fitpotA = func(xDataA, prAntes[0], prAntes[1])
        #fitpotB = func(xDataB, prDespues[0], prDespues[1])


        exp.dinamica['expotDespues'] = prDespues[0][0]
        exp.dinamica['ctepotDespues'] = prDespues[0][1]

In [30]:
#[LISTO] Lista para poder plotear con los valores de antes y dp L*,t*

listaajustedoscurvas = []

for exp in Experimentos:
    try:
        holder = {}
        holder['caudal'] = exp.param['caudal'][0]
        holder['angulo'] = exp.param['angulo inclinacion'][0]#en grados
        holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])
        holder['temperatura'] = exp.param["t max"][0]
        holder['v1'] = exp.param['v1'][0]    
        holder['v2'] = exp.param['v2'][0]
        holder['t*'] = exp.param['t*'][0]
        holder['L*'] = exp.param['L*'][0]
        holder['nombre'] = exp.param['NOMBRE'][0]
        holder['xData'] = exp.dinamica['avance: tiempo']
        holder['yData'] = exp.dinamica['avance: distancia desde punto eyeccion']
        holder['yDataymed'] = exp.dinamica['avance: ancho maximo']/2
        holder['yDataymax'] = exp.dinamica['avance: ancho maximo']
        holder['yDataxmin'] = exp.dinamica['avance: distancia hacia atrás']
        if 'yDataAntes' in exp.dinamica.columns:
            if exp.dinamica['yDataAntes'].shape[0] >= 2:
                holder['cteA'] = exp.dinamica['cteAntes'][0]
                holder['expA'] = exp.dinamica['expAntes'][0]
                holder['xDataA'] = exp.dinamica['xDataAntes']
                holder['yDataA'] = exp.dinamica['yDataAntes']
        if 'yDataDespues' in exp.dinamica.columns:
            if exp.dinamica['yDataDespues'].shape[0] >= 2:
                holder['cteD'] = exp.dinamica['cteDespues'][0]
                holder['expD'] = exp.dinamica['expDespues'][0]
                holder['xDataD'] = exp.dinamica['xDataDespues']
                holder['yDataD'] = exp.dinamica['yDataDespues']

        listaajustedoscurvas.append(holder)
    except Exception as e:
        print(exp, e)
        print(exp.dinamica['yDataAntes'])        
        print(exp.dinamica['yDataDespues'])
        continue

In [42]:
#Ploteo de datos por caudal para ajuste de dos curvas con datos ploteados.
#plot listafuncionespotlog todos los exp de 1 caudal a la vez
# 1 cc 10 exp (-1 1cc48C) 5 cc 8 exp 10cc 1e-05 8 exp 25cc 2.5e-05 6 exp 5a25xx 1 exp
    
fig = plt.figure()
axes = []
f = 0

for x in range(9): #range segun el numero de exp por caudal
    axes.append(fig.add_subplot(3, 3, x + 1)) #subplot segun numero de figuras en el plot (3x3, 4x3, 2x3 etc)

for i, (key, group) in enumerate(groupby(sorted(listaajustedoscurvas, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(i, key)
    if key==1e-06:
        for exp in sorted(group, key=lambda x: x['temperatura']):
            
            #excepciones
            if exp['nombre'] == '1cc48C':
                continue
                
            #obtencion de datos
            fin = exp['xData'].iloc[-1]
            dist = exp['yData'].iloc[-1]
            
            if 'cteA' in exp:
                cteA = exp['cteA']
                expA = exp['expA']
                
            if 'cteD' in exp:
                cteD = exp['cteD'] 
                expD = exp['expD'] 

            axes[f].set_title(exp['nombre'])
            
            textstr = '$\star k_a=%.3f$ \n$c_a=%.3f$ \n$\star k_d=%.3f$ \n$c_d=%.3f$'%(expA,cteA,expD,cteD)

            axes[f].set_ylabel('log(Distancia [m])')
            axes[f].set_xlabel('log(Tiempo [s])')
            #axes[f].set_title('{}'.format(exp['nombre']))
            axes[f].text(fin/10, dist*0.065, textstr)
            axes[f].grid(True, which='both', color='gainsboro')
            

            if exp['angulo'] == 15:
                axes[f].loglog(exp['xData'], exp['yData'], color='m', marker='x', 
                               markersize='4', label=r'$X_(max)$')
                axes[f].loglog(exp['xData'], exp['yDataymed'], color='g', linestyle='-.', label=r'$Y_(max)$')
                axes[f].loglog(exp['xData'], exp['yDataxmin'], color='g', linestyle='--', label=r'$X_(min)$')
                if 'cteA' in exp:
                    axes[f].loglog(exp['xDataA'][1:], cteA*exp['xDataA'][1:]**expA, 
                                   color='b', linestyle=':', label='Ajuste teórico logaritmico')
                #plt.loglog(xDataA[1:], cA*xDataA[1:]**kA, color='g', linestyle=':', label='Ajuste teórico potencial')
                if 'cteD' in exp:    
                    axes[f].loglog(exp['xDataD'], cteD*exp['xDataD']**expD, color='r', 
                                   linestyle=':', label='Ajuste teórico logaritmico')
                #plt.loglog(xDataD, cB*xDataD**kB, color='c', linestyle=':', label='Ajuste teórico potencial')

            else:
                axes[f].loglog(exp['xData'], exp['yData'], color='m', marker='o', 
                               markersize='4', label='Datos Experimentales')
                axes[f].loglog(exp['xData'], exp['yDataymed'], color='g', linestyle='-.', label=r'$Y_(max)$')
                axes[f].loglog(exp['xData'], exp['yDataxmin'], color='g', linestyle='--', label=r'$X_(min)$')

                if 'cteA' in exp:
                    axes[f].loglog(exp['xDataA'][1:], cteA*exp['xDataA'][1:]**expA, 
                                   color='b', linestyle='-.', label='Ajuste teórico logaritmico')
                #plt.loglog(xDataA[1:], cA*xDataA[1:]**kA, color='g', linestyle=':', label='Ajuste teórico potencial')
                if 'cteD' in exp:
                    axes[f].loglog(exp['xDataD'], cteD*exp['xDataD']**expD, 
                                   color='r', linestyle='-.', label='Ajuste teórico logaritmico')
                #plt.loglog(xDataD, cB*xDataD**kB, color='c', linestyle=':', label='Ajuste teórico potencial')

            f = f+1
            print(f)
            #print(i, exp['nombre'])        

    plt.legend(handles=[datos, ajustelog, ecuacion1, ajustepot, ecuacion2], title='Leyenda', loc=1, 
               bbox_to_anchor=(1.55, 0.4, 0.4, 0.4), labels=['Datos Experimentales', 'Ajuste pre L*,t*',r'$c_a*t^k_a$', 
                                                            'Ajuste Teorico post L*,t*', r'$c_d*t^k_d$'], ncol=1)
    plt.suptitle('Ajuste teórico caudales 1[cc/s]')
    fig.show()

0 1e-06
1
2
3
4
5
6
7
8
9
1 5e-06
2 1e-05
3 1.7e-05
4 2.5e-05
